In [34]:
# imports
import typing
from prefect import task, flow
from prefect_gcp import GcpCredentials
from google.cloud import bigquery

print("Setup Complete")

Setup Complete


In [35]:
# Deployment 2
@task(log_prints=True, name="get-gcp-creds")
# Define a function to get GCP Credentials
def get_bigquery_client():
    gcp_creds_block = GcpCredentials.load(
        "prefect-gcs-2023-creds"
    )  # Change this credentials to yours
    gcp_creds = gcp_creds_block.get_credentials_from_service_account()
    client = bigquery.Client(credentials=gcp_creds)
    return client

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:270: UserWarning: A task named 'get-gcp-creds' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_24489/4149454993.py:2' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [36]:
# @flow()
# def create_dataset(dataset_name: str):
#     # Construct a BigQuery client object.
#     client = get_bigquery_client()

#     # TODO(developer): Set dataset_id to the ID of the dataset to create.
#     dataset_id = f"{client.project}.{dataset_name}"

#     # Construct a full Dataset object to send to the API.
#     dataset = bigquery.Dataset(dataset_id)

#     # TODO(developer): Specify the geographic location where the dataset should reside.
#     dataset.location = "asia-southeast1"

#     # Send the dataset to the API for creation, with an explicit timeout.
#     # Raises google.api_core.exceptions.Conflict if the Dataset already
#     # exists within the project.
#     dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
#     print(f"Created dataset {client.project}.{dataset.dataset_id}")
#     return

# create_dataset(dataset_name ="prod_bandcamp")

In [37]:
# Upload data from GCS to BigQuery
@flow(log_prints=True, name="etl-gcs-to-bq")
def etl_gcs_to_bq(file_num: int):
    client = get_bigquery_client()
    table_id = f"dtc-de-2023.stg_bandcamp.albums-full-info-{file_num}"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,
        schema=[
            bigquery.SchemaField("_id", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("numTracks", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("keywords", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("datePublished", "DATETIME", mode="NULLABLE"),
            bigquery.SchemaField("name", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("dateModified", "DATETIME", mode="NULLABLE"),
            bigquery.SchemaField("comment", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("inAlbum", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("offers", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("duration_secs", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("url", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("duration", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("isrcCode", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("byArtist_image", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("byArtist_genre", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("byArtist_@type", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("byArtist_sameAs", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("byArtist_name", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("byArtist_description", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("track_itemListElement", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("track_@type", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("track_numberOfItems", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("track", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("inAlbum_name", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("inAlbum_@id", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("inAlbum_@type", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("offers_availability", "STRING", mode="NULLABLE"),
            bigquery.SchemaField(
                "offers_priceSpecification_minPrice", "FLOAT", mode="NULLABLE"
            ),
            bigquery.SchemaField("offers_price", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("offers_@type", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("offers_priceCurrency", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("offers_url", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("recordingOf_@type", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("recordingOf_lyrics_text", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("recordingOf_lyrics_@type", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("byArtist", "FLOAT", mode="NULLABLE"),
        ],
    )
    uri = f"gs://prefect-gcs-bucket-bandcamp/albums-full-info-{file_num}"

    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)
    print(f"Loaded {destination_table.num_rows} rows.")


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:214: UserWarning: A flow named 'etl-gcs-to-bq' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_24489/693090197.py:2' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [38]:
@flow(log_prints=True, name="deduplicate data")
# Define a function to remove duplicate
def deduplicate_data(num: int):
    client = get_bigquery_client()

    query_dedup = f"CREATE OR REPLACE TABLE \
                        `dtc-de-2023.stg_bandcamp.albums-full-info-{num}`  AS ( \
                            SELECT DISTINCT * \
                            FROM `dtc-de-2023.stg_bandcamp.albums-full-info-{num}` \
                            )"

    # limit query to 10GB
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
    # priority=bigquery.QueryPriority.BATCH
    # query
    query_job = client.query(query_dedup, job_config=safe_config)

    # Check progress
    query_job = typing.cast(
        "bigquery.QueryJob",
        client.get_job(
            query_job.job_id, location=query_job.location
        ),  # Make an API request.
    )
    print("Complete removing duplicates")
    print(f"Job {query_job.job_id} is currently in state {query_job.state}")


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:214: UserWarning: A flow named 'deduplicate data' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_24489/895334318.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [39]:
# TABLE_LIST = [f"albums-full-info-{file_num}" for file_num in range(1,12)] 
# TABLE_LIST

In [40]:
# Union all tables
def union_all():
    # Parameters
    client = get_bigquery_client()
    project_id = "dtc-de-2023"
    dataset_id = "stg_bandcamp"
    table_list = [f"albums-full-info-{file_num}" for file_num in range(1,12)]
    # Query
    query_template = "SELECT * FROM `{project_id}.{dataset_id}.{table_name}`"
    union_table_name = "bandcamp_items_all"
    destination_table_id = f"{project_id}.{dataset_id}.{union_table_name}"
    
    union_query = "\nUNION ALL\n".join([query_template.format(project_id=project_id, dataset_id=dataset_id, table_name=table) for table in table_list])
    
    job_config = bigquery.QueryJobConfig(destination=destination_table_id, write_disposition="WRITE_TRUNCATE")
    job = client.query(query=union_query, job_config=job_config)
    job.result
    return


In [41]:
# Parent flow ETL
@flow(log_prints=True, name="etl-parent-to-bq")
def etl_parent_bq_flow(file_num_list: list[int] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]):
    for file_num in file_num_list:
        etl_gcs_to_bq(file_num)
    # Union all tables
    union_all()

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:214: UserWarning: A flow named 'etl-parent-to-bq' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_24489/748791523.py:2' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [42]:
# Run main
if __name__ == "__main__":
    file_num_list = [10,11]

    etl_parent_bq_flow(file_num_list)

12:04:58.113 | INFO    | prefect.engine - Created flow run 'aquamarine-echidna' for flow 'etl-parent-to-bq'

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:270: UserWarning: A task named 'etl-gcs-to-bq' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_24489/693090197.py:2' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


12:05:01.735 | INFO    | Flow run 'aquamarine-echidna' - Created subflow run 'wooden-parakeet' for flow 'etl-gcs-to-bq'

12:05:02.806 | INFO    | Flow run 'wooden-parakeet' - Created task run 'get-gcp-creds-0' for task 'get-gcp-creds'

12:05:02.809 | INFO    | Flow run 'wooden-parakeet' - Executing 'get-gcp-creds-0' immediately...

...

12:05:16.664 | ERROR   | Flow run 'wooden-parakeet' - Encountered exception during execution:

12:05:17.192 | ERROR   | Flow run 'wooden-parakeet' - Finished in state Failed('Flow run encountered an exception. google.api_core.exceptions.BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/dtc-de-2023/jobs?prettyPrint=false: Invalid field name "byArtist_@type". Fields must contain the allowed characters, and be at most 300 characters long. For allowed characters, please refer to https://cloud.google.com/bigquery/docs/schemas#column_names\n')

12:05:17.201 | ERROR   | Flow run 'aquamarine-echidna' - Encountered exception during execution:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 665, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_24489/748791523.py", line 5, in etl_parent_bq_flow
    etl_gcs_to_bq(file_num)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py", line 468, in __call__
    return enter_flow_run_engine_from_flow_call(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 179, in enter_flow_run_engine_from_flow_call
    return run_async_from_worker_thread(begin_run)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 177, in run_async_from_worker_thread
    return anyio.from_thread.run(call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/from_thread.py", line 49, in run
    return asynclib.run_async_from_thread(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 970, in run_async_from_thread
    return f.result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 446, in result
    return self.__get_result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 391, in __get_result
    raise self._exception
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/client/utilities.py", line 47, in with_injected_client
    return await fn(*args, **kwargs)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 568, in create_and_begin_subflow_run
    return await terminal_state.result(fetch=True)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/states.py", line 91, in _get_state_result
    raise await get_state_exception(state)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 665, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File

12:05:17.583 | ERROR   | Flow run 'aquamarine-echidna' - Finished in state Failed('Flow run encountered an exception. google.api_core.exceptions.BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/dtc-de-2023/jobs?prettyPrint=false: Invalid field name "byArtist_@type". Fields must contain the allowed characters, and be at most 300 characters long. For allowed characters, please refer to https://cloud.google.com/bigquery/docs/schemas#column_names\n')

BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/dtc-de-2023/jobs?prettyPrint=false: Invalid field name "byArtist_@type". Fields must contain the allowed characters, and be at most 300 characters long. For allowed characters, please refer to https://cloud.google.com/bigquery/docs/schemas#column_names